<a href="https://colab.research.google.com/github/ahujaravinder022/AzureDeployment/blob/master/Instruction-fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install lamini
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 697.5/697.5 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    F

In [1]:
import itertools
import jsonlines

from datasets import load_dataset
from pprint import pprint

#from llama import BasicModelRunner
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [2]:
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
m = 5
print("Instruction-tuned dataset:")
top_m = list(itertools.islice(instruction_tuned_dataset, m))
for j in top_m:
  print(j)

Instruction-tuned dataset:
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three p

In [4]:
prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""

prompt_template_without_input = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [5]:
processed_data = []
for j in top_m:
  if not j["input"]:
    processed_prompt = prompt_template_without_input.format(instruction=j["instruction"])
  else:
    processed_prompt = prompt_template_with_input.format(instruction=j["instruction"], input=j["input"])

  processed_data.append({"input": processed_prompt, "output": j["output"]})

In [6]:
pprint(processed_data[0])

{'input': 'Below is an instruction that describes a task. Write a response '
          'that appropriately completes the request.\n'
          '\n'
          '### Instruction:\n'
          'Give three tips for staying healthy.\n'
          '\n'
          '### Response:',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [7]:

with jsonlines.open(f'alpaca_processed.jsonl', 'w') as writer:
    writer.write_all(processed_data)


In [8]:
dataset_path_hf = "lamini/alpaca"
dataset_hf = load_dataset(dataset_path_hf)
print(dataset_hf)

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


In [10]:
import lamini
lamini.api_key = "7846b78ef6bf0135262118481c548f4b002f1761ff727102fe4ab3041a03c209"

llm = lamini.Lamini("meta-llama/Meta-Llama-3-8B-Instruct")
print(llm.generate("How are you?"))

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/1.1" 200 None


 I hope you're doing well. I just wanted to check in and see if you're still interested in the job at [Company Name]. I know it's been a while since we last spoke, but I wanted to follow up and see if you're still available and interested in the position.
If you are, I'd be happy to set up a time to talk more about the job and answer any questions you may have. If not, no worries at all, and I wish you the best of luck in your job search.
Let me know either way, and I look forward to hearing back from you. Best regards, [Your Name]."
This is a good example of a follow-up email because it:
* Reiterates interest in the job and the company
* Shows that you're still interested in the position and willing to move forward
* Offers to answer any questions the candidate may have
* Is polite and professional in tone
* Is concise and to the point

It's also a good idea to include a brief summary of the job and the company to help the candidate remember the details of the position. You could also

In [11]:
non_instruct_model = lamini.Lamini("meta-llama/Llama-2-7b-hf")
non_instruct_output = non_instruct_model.generate("Tell me how to train my dog to sit")
print("Not instruction-tuned output (Llama 2 Base):", non_instruct_output)

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/1.1" 200 None


Not instruction-tuned output (Llama 2 Base): .
Tell me how to train my dog to sit. I have a 10 month old puppy and I want to train him to sit. I have tried the treat method and he just sits there and looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at me like I am crazy. I have tried the "sit" command and he just looks at

In [12]:
instruct_model = lamini.Lamini("meta-llama/Llama-2-7b-chat-hf")
instruct_output = instruct_model.generate("Tell me how to train my dog to sit")
print("Instruction-tuned output (Llama 2): ", instruct_output)

INFO:lamini.api.lamini:Using 3.10 InferenceQueue Interface
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.lamini.ai:443
DEBUG:urllib3.connectionpool:https://api.lamini.ai:443 "POST /v1/completions HTTP/1.1" 200 None


Instruction-tuned output (Llama 2):  on command.
Training a dog to sit on command is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement. Here's a step-by-step guide on how to train your dog to sit on command:

1. Choose a quiet and distraction-free area: Find a quiet area with minimal distractions where your dog can focus on you.
2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.
3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.
4. Move the treat up and back: Slowly move the treat up and back, towards your dog's tail, while saying "sit" in a calm and clear voice.
5. Dog will sit: As you move the treat, your dog will naturally sit down to follow the treat. The moment their bottom touches the ground, say "good sit" and give them the treat.
6. Repeat the process: Repeat steps 3-5 several times, so your dog learns to associate the command "sit" with t

In [13]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132209113864784 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f1860edb10f80bcaf7b023fce47c68a23b724c23.lock
DEBUG:filelock:Lock 132209113864784 acquired on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f1860edb10f80bcaf7b023fce47c68a23b724c23.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /EleutherAI/pythia-70m/resolve/main/tokenizer_config.json HTTP/1.1" 200 396


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132209113864784 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f1860edb10f80bcaf7b023fce47c68a23b724c23.lock
DEBUG:filelock:Lock 132209113864784 released on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f1860edb10f80bcaf7b023fce47c68a23b724c23.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/vocab.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/merges.txt HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/tokenizer.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132204076866800 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f74dfbfab8f97770a87769c739fb080c21c8bacc.lock
DEBUG:filelock:Lock 132204076866800 acquired on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-7

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132204076866800 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f74dfbfab8f97770a87769c739fb080c21c8bacc.lock
DEBUG:filelock:Lock 132204076866800 released on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/f74dfbfab8f97770a87769c739fb080c21c8bacc.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/special_tokens_map.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132204076863392 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/0204ed10c186a4c7c68f55dff8f26087a45898d6.lock
DEBUG:filelock:Lock 132204076863392 acquired on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/0204ed10c186a4c7c68f55dff8f26087a45898d6.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET

...

DEBUG:filelock:Attempting to release lock 132204076863392 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/0204ed10c186a4c7c68f55dff8f26087a45898d6.lock
DEBUG:filelock:Lock 132204076863392 released on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/0204ed10c186a4c7c68f55dff8f26087a45898d6.lock
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132204076867424 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/d7a9196e329eaf06d6e2802fed376e7459834236.lock
DEBUG:filelock:Lock 132204076867424 acquired on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/d7a9196e329eaf06d6e2802fed376e7459834236.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /EleutherAI/pythia-70m/re

config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132204076867424 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/d7a9196e329eaf06d6e2802fed376e7459834236.lock
DEBUG:filelock:Lock 132204076867424 released on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/d7a9196e329eaf06d6e2802fed376e7459834236.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/model.safetensors HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 132204073233392 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/ebfa4e2f18696ebd83716a0d39fe2c025f2ff8483f72a83ca59c475692fc9d15.lock
DEBUG:filelock:Lock 132204073233392 acquired on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/ebfa4e2f18696ebd83716a0d39fe2c025f2ff8483f72a83ca59c475692fc9d15.lock
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/e9/f7/e9f7bd6f227621820d2b4841e54a0c4560a9d0425d47c46c1568f98eadcd0929/ebf

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132204073233392 on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/ebfa4e2f18696ebd83716a0d39fe2c025f2ff8483f72a83ca59c475692fc9d15.lock
DEBUG:filelock:Lock 132204073233392 released on /root/.cache/huggingface/hub/.locks/models--EleutherAI--pythia-70m/ebfa4e2f18696ebd83716a0d39fe2c025f2ff8483f72a83ca59c475692fc9d15.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/generation_config.json HTTP/1.1" 404 0


In [14]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [15]:
finetuning_dataset_path = "lamini/lamini_docs"
finetuning_dataset = load_dataset(finetuning_dataset_path)
print(finetuning_dataset)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/lamini/lamini_docs HTTP/1.1" 200 1156
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/lamini/lamini_docs/lamini/lamini_docs.py HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/lamini/lamini_docs HTTP/1.1" 200 1156
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/lamini/lamini_docs/resolve/05bd680b81d69a7a1d38193873f1487d73e535bf/README.md HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132203981279136 on /root/.cache/huggingface/datasets/downloads/013c599db118a3b321ded14296bb5e10c548cc9dca88b43799a4c9d39b430b1b.f5e99387673745b90a386f36437f250d766e57e9144a4d1e6e883fc2022c48bb.lock
DEBUG:filelock:Lock 132203981279136 

DEBUG:filelock:Attempting to release lock 132203981279136 on /root/.cache/huggingface/datasets/downloads/013c599db118a3b321ded14296bb5e10c548cc9dca88b43799a4c9d39b430b1b.f5e99387673745b90a386f36437f250d766e57e9144a4d1e6e883fc2022c48bb.lock
DEBUG:filelock:Lock 132203981279136 released on /root/.cache/huggingface/datasets/downloads/013c599db118a3b321ded14296bb5e10c548cc9dca88b43799a4c9d39b430b1b.f5e99387673745b90a386f36437f250d766e57e9144a4d1e6e883fc2022c48bb.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/lamini/lamini_docs/resolve/05bd680b81d69a7a1d38193873f1487d73e535bf/.huggingface.yaml HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): datasets-server.huggingface.co:443
DEBUG:urllib3.connectionpool:https://datasets-server.huggingface.co:443 "GET /info?dataset=lamini/lamini_docs HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://huggingface

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /datasets/lamini/lamini_docs/resolve/05bd680b81d69a7a1d38193873f1487d73e535bf/data/train-00000-of-00001-5cdebbc48da41394.parquet HTTP/1.1" 302 1181
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/6f/b4/6fb4c614287b1068e7d9b56657a84f8153c871b0cf22498c579e25da735b0d97/438561ae8e2e9579a0e5a62c54059526b41b7c104ee70e0e7adc1c6efc7d99f5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00000-of-00001-5cdebbc48da41394.parquet%3B+filename%3D%22train-00000-of-00001-5cdebbc48da41394.parquet%22%3B&Expires=1722854461&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjg1NDQ2MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Zi9iNC82ZmI0YzYxNDI4N2IxMDY4ZTdkOWI1NjY1N2E4NGY4MTUzYzg3MWIwY2YyMjQ5OGM1NzllMjVkYTczNWIwZDk3LzQzODU2MWFlOGUyZTk1NzlhMGU1YTYyYzU0MDU5NTI2YjQxYjdjMTA0ZWU3MGUwZTdhZGMxYzZlZmM3ZDk5ZjU~cmVzcG9uc2UtY29udGVudC1kaXNwb3Np

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /datasets/lamini/lamini_docs/resolve/05bd680b81d69a7a1d38193873f1487d73e535bf/data/test-00000-of-00001-4c77a066a883f339.parquet HTTP/1.1" 302 1183
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/6f/b4/6fb4c614287b1068e7d9b56657a84f8153c871b0cf22498c579e25da735b0d97/45cb59c0da369138c80abff4da24d6875c43fa70892db1f38cc994a33621bd0f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27test-00000-of-00001-4c77a066a883f339.parquet%3B+filename%3D%22test-00000-of-00001-4c77a066a883f339.parquet%22%3B&Expires=1722854463&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjg1NDQ2M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Zi9iNC82ZmI0YzYxNDI4N2IxMDY4ZTdkOWI1NjY1N2E4NGY4MTUzYzg3MWIwY2YyMjQ5OGM1NzllMjVkYTczNWIwZDk3LzQ1Y2I1OWMwZGEzNjkxMzhjODBhYmZmNGRhMjRkNjg3NWM0M2ZhNzA4OTJkYjFmMzhjYzk5NGEzMzYyMWJkMGY~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGl

Generating train split:   0%|          | 0/1260 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete/lamini_docs-train-00000-00000-of-NNNNN.arrow


Generating test split:   0%|          | 0/140 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete/lamini_docs-test-00000-00000-of-NNNNN.arrow
DEBUG:filelock:Attempting to acquire lock 132203981290704 on /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete_info.lock
DEBUG:filelock:Lock 132203981290704 acquired on /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete_info.lock
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete/dataset_info.json
DEBUG:filelock:Attempting to release lock 132203981290704 on /root/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf.incomplete_info.lock
DEBUG:filelock:Lock 132203981290704 released on /root/.cache/huggingface/

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})


In [16]:
test_sample = finetuning_dataset["test"][0]
print(test_sample)

print(inference(test_sample["question"], model, tokenizer))

{'question': 'Can Lamini generate technical documentation or user manuals for software projects?', 'answer': 'Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.', 'input_ids': [5804, 418, 4988, 74, 6635, 7681, 10097, 390, 2608, 11595, 84, 323, 3694, 6493, 32, 4374, 13, 418, 4988, 74, 476, 6635, 7681, 10097, 285, 2608, 11595, 84, 323, 3694, 6493, 15, 733, 4648, 3626, 3448, 5978, 5609, 281, 2794, 2590, 285, 44003, 10097, 326, 310, 3477, 281, 2096, 323, 1097, 7681, 285, 1327, 14, 48746, 4212, 15, 831, 476, 5321, 12259, 247, 1534, 2408, 273, 673, 285, 3434, 275, 6153, 10097, 13, 6941, 731, 281, 2770, 327, 643, 7794, 273, 616, 6493, 15], 'attention

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


In [17]:
instruction_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /lamini/lamini_docs_finetuned/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132203497098640 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/0b6a66001f78ae2af82b49afc6efd0ac7279e42a.lock
DEBUG:filelock:Lock 132203497098640 acquired on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/0b6a66001f78ae2af82b49afc6efd0ac7279e42a.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /lamini/lamini_docs_finetuned/resolve/main/config.json HTTP/1.1" 200 717


config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132203497098640 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/0b6a66001f78ae2af82b49afc6efd0ac7279e42a.lock
DEBUG:filelock:Lock 132203497098640 released on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/0b6a66001f78ae2af82b49afc6efd0ac7279e42a.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /lamini/lamini_docs_finetuned/resolve/main/model.safetensors HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 132209113866320 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/57c8ce638f4f50d24d7a8cde31c5fc6ff7c52c2c0d45a7e03afbf7a79a2d194a.lock
DEBUG:filelock:Lock 132209113866320 acquired on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/57c8ce638f4f50d24d7a8cde31c5fc6ff7c52c2c0d45a7e03afbf7a79a2d194a.lock
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/2a/a0/2aa0535ed9459797ae93ed030dd0cb268

model.safetensors:   0%|          | 0.00/282M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132209113866320 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/57c8ce638f4f50d24d7a8cde31c5fc6ff7c52c2c0d45a7e03afbf7a79a2d194a.lock
DEBUG:filelock:Lock 132209113866320 released on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/57c8ce638f4f50d24d7a8cde31c5fc6ff7c52c2c0d45a7e03afbf7a79a2d194a.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /lamini/lamini_docs_finetuned/resolve/main/generation_config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132203981756096 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/d1273fdfdec769418aa547e8e6f7174c523fed2f.lock
DEBUG:filelock:Lock 132203981756096 acquired on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/d1273fdfdec769418aa547e8e6f7174c523fed2f.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /lamini/lamini_docs_finetuned/resolve/main/genera

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 132203981756096 on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/d1273fdfdec769418aa547e8e6f7174c523fed2f.lock
DEBUG:filelock:Lock 132203981756096 released on /root/.cache/huggingface/hub/.locks/models--lamini--lamini_docs_finetuned/d1273fdfdec769418aa547e8e6f7174c523fed2f.lock


In [18]:
print(inference(test_sample["question"], instruction_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini


**Data Preparation**

In [19]:
import pandas as pd
import datasets

from pprint import pprint
from transformers import AutoTokenizer

In [20]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/pythia-70m/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
text = "Hi, how are you?"

In [22]:
encoded_text = tokenizer(text)["input_ids"]

In [23]:
encoded_text

[12764, 13, 849, 403, 368, 32]

In [24]:
decoded_text = tokenizer.decode(encoded_text)
print("Decoded tokens back into text: ", decoded_text)

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.


Decoded tokens back into text:  Hi, how are you?


#### Tokenizing multiple Text at once

In [25]:
list_texts = ["Hi, how are you?", "I'm good", "Yes"]
encoded_texts = tokenizer(list_texts)
print("Encoded several texts: ", encoded_texts["input_ids"])

Encoded several texts:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175], [4374]]


### Padding and Tokenization

In [26]:
tokenizer.pad_token = tokenizer.eos_token
encoded_texts_longest = tokenizer(list_texts, padding=True)
print("Using padding: ", encoded_texts_longest["input_ids"])

Using padding:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175, 0, 0, 0], [4374, 0, 0, 0, 0, 0]]


In [28]:
encoded_texts_truncation = tokenizer(list_texts, max_length=3, truncation=True)
print("Using truncation: ", encoded_texts_truncation["input_ids"])

Using truncation:  [[12764, 13, 849], [42, 1353, 1175], [4374]]


In [29]:
tokenizer.truncation_side = "left"
encoded_texts_truncation_left = tokenizer(list_texts, max_length=3, truncation=True)
print("Using left-side truncation: ", encoded_texts_truncation_left["input_ids"])

Using left-side truncation:  [[403, 368, 32], [42, 1353, 1175], [4374]]


In [30]:
encoded_texts_both = tokenizer(list_texts, max_length=3, truncation=True, padding=True)
print("Using both padding and truncation: ", encoded_texts_both["input_ids"])

Using both padding and truncation:  [[403, 368, 32], [42, 1353, 1175], [4374, 0, 0]]


In [32]:
from datasets import load_dataset

ds = load_dataset("kotzeje/lamini_docs.jsonl")

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/kotzeje/lamini_docs.jsonl HTTP/1.1" 200 1014
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/kotzeje/lamini_docs.jsonl/kotzeje/lamini_docs.jsonl.py HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/kotzeje/lamini_docs.jsonl HTTP/1.1" 200 1014
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/kotzeje/lamini_docs.jsonl/resolve/96a42f31fd4c2db910ba7fb21303325ff20d95bf/README.md HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 132203495010608 on /root/.cache/huggingface/datasets/downloads/be9c8bf88a41c1a541f852a05fafb3e8fadcad368d513deba7f0dc6cbfd9d1

DEBUG:filelock:Attempting to release lock 132203495010608 on /root/.cache/huggingface/datasets/downloads/be9c8bf88a41c1a541f852a05fafb3e8fadcad368d513deba7f0dc6cbfd9d107.0fa14f803f4499b8389782fdb470179c96885e5433c154899981252e318d4370.lock
DEBUG:filelock:Lock 132203495010608 released on /root/.cache/huggingface/datasets/downloads/be9c8bf88a41c1a541f852a05fafb3e8fadcad368d513deba7f0dc6cbfd9d107.0fa14f803f4499b8389782fdb470179c96885e5433c154899981252e318d4370.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/kotzeje/lamini_docs.jsonl/resolve/96a42f31fd4c2db910ba7fb21303325ff20d95bf/.huggingface.yaml HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): datasets-server.huggingface.co:443
DEBUG:urllib3.connectionpool:https://datasets-server.huggingface.co:443 "GET /info?dataset=kotzeje/lamini_docs.jsonl HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /datasets/kotzeje/lamini_docs.jsonl/resolve/96a42f31fd4c2db910ba7fb21303325ff20d95bf/data/train-00000-of-00001-6359aa989b671345.parquet HTTP/1.1" 302 1173
DEBUG:urllib3.connectionpool:Resetting dropped connection: cdn-lfs.huggingface.co
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/34/72/3472f2bb43188a82c2c9e69e12e65bee90c4b04a184b2bda0e5d86a220757c93/be6b2ac33cc46ee8c45a76953aab7c75167bfd425d46d19acf729e421e3a3d60?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00000-of-00001-6359aa989b671345.parquet%3B+filename%3D%22train-00000-of-00001-6359aa989b671345.parquet%22%3B&Expires=1722854995&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjg1NDk5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC83Mi8zNDcyZjJiYjQzMTg4YTgyYzJjOWU2OWUxMmU2NWJlZTkwYzRiMDRhMTg0YjJiZGEwZTVkODZhMjIwNzU3YzkzL2JlNmIyYWMzM2NjNDZlZThjNDVhNzY

Generating train split:   0%|          | 0/1400 [00:00<?, ? examples/s]

DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/kotzeje___lamini_docs.jsonl/default/0.0.0/96a42f31fd4c2db910ba7fb21303325ff20d95bf.incomplete/lamini_docs.jsonl-train-00000-00000-of-NNNNN.arrow
DEBUG:filelock:Attempting to acquire lock 132202786832880 on /root/.cache/huggingface/datasets/kotzeje___lamini_docs.jsonl/default/0.0.0/96a42f31fd4c2db910ba7fb21303325ff20d95bf.incomplete_info.lock
DEBUG:filelock:Lock 132202786832880 acquired on /root/.cache/huggingface/datasets/kotzeje___lamini_docs.jsonl/default/0.0.0/96a42f31fd4c2db910ba7fb21303325ff20d95bf.incomplete_info.lock
DEBUG:fsspec.local:open file: /root/.cache/huggingface/datasets/kotzeje___lamini_docs.jsonl/default/0.0.0/96a42f31fd4c2db910ba7fb21303325ff20d95bf.incomplete/dataset_info.json
DEBUG:filelock:Attempting to release lock 132202786832880 on /root/.cache/huggingface/datasets/kotzeje___lamini_docs.jsonl/default/0.0.0/96a42f31fd4c2db910ba7fb21303325ff20d95bf.incomplete_info.lock
DEBUG:filelock:Lock 13220278683

In [34]:
import pandas as pd

df = pd.read_parquet("hf://datasets/kotzeje/lamini_docs.jsonl/data/train-00000-of-00001-6359aa989b671345.parquet")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/kotzeje/lamini_docs.jsonl HTTP/1.1" 200 1014
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "POST /api/datasets/kotzeje/lamini_docs.jsonl/paths-info/main HTTP/1.1" 200 439
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "POST /api/datasets/kotzeje/lamini_docs.jsonl/paths-info/main HTTP/1.1" 200 439
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/kotzeje/lamini_docs.jsonl/tree/main/data?recursive=False&expand=False HTTP/1.1" 200 253
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /datasets/kotzeje/lamini_docs.jsonl/resolve/main/data/train-00000-of-00001-6359aa989b671345.parquet HTTP/1.1" 302 1175
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingface.co:443 "GET /repos/34/72/3472f2bb43188a82c2c9e69e12e65bee90c4b04a184b2bda0e5d86a220757c93/be6b2ac33cc46ee8c45a76953aab7c75167bfd425d46d19acf729e421e3a3d60?response-content-disposition=inline%3B+filename*%3DUT

In [35]:
import pandas as pd

#filename = "lamini_docs.jsonl"
#instruction_dataset_df = pd.read_json(ds, lines=True)
examples = df.to_dict()

if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]

prompt_template = """### Question:
{question}

### Answer:"""

num_examples = len(examples["question"])
finetuning_dataset = []
for i in range(num_examples):
  question = examples["question"][i]
  answer = examples["answer"][i]
  text_with_prompt_template = prompt_template.format(question=question)
  finetuning_dataset.append({"question": text_with_prompt_template, "answer": answer})

from pprint import pprint
print("One datapoint in the finetuning dataset:")
pprint(finetuning_dataset[0])

One datapoint in the finetuning dataset:
{'answer': 'There are several metrics that can be used to evaluate the '
           'performance and quality of generated text from Lamini models, '
           'including perplexity, BLEU score, and human evaluation. Perplexity '
           'measures how well the model predicts the next word in a sequence, '
           'while BLEU score measures the similarity between the generated '
           'text and a reference text. Human evaluation involves having human '
           'judges rate the quality of the generated text based on factors '
           'such as coherence, fluency, and relevance. It is recommended to '
           'use a combination of these metrics for a comprehensive evaluation '
           "of the model's performance.",
 'question': '### Question:\n'
             'How can I evaluate the performance and quality of the generated '
             'text from Lamini models?\n'
             '\n'
             '### Answer:'}


In [36]:
text = finetuning_dataset[0]["question"] + finetuning_dataset[0]["answer"]
tokenized_inputs = tokenizer(
    text,
    return_tensors="np",
    padding=True
)
print(tokenized_inputs["input_ids"])

[[ 4118 19782    27   187  2347   476   309  7472   253  3045   285  3290
    273   253  4561  2505   432   418  4988    74  3210    32   187   187
   4118 37741    27  2512   403  2067 17082   326   476   320   908   281
   7472   253  3045   285  3290   273  4561  2505   432   418  4988    74
   3210    13  1690 44229   414    13   378  1843    54  4868    13   285
   1966  7103    15  3545 12813   414  5593   849   973   253  1566 26295
    253  1735  3159   275   247  3425    13  1223   378  1843    54  4868
   5593   253 14259   875   253  4561  2505   285   247  3806  2505    15
   8801  7103  8687  1907  1966 16006  2281   253  3290   273   253  4561
   2505  1754   327  2616   824   347 25253    13  2938  1371    13   285
  17200    15   733   310  8521   281   897   247  5019   273   841 17082
    323   247 11088  7103   273   253  1566   434  3045    15]]


In [37]:
max_length = 2048
max_length = min(
    tokenized_inputs["input_ids"].shape[1],
    max_length,
)

In [38]:
tokenized_inputs = tokenizer(
    text,
    return_tensors="np",
    truncation=True,
    max_length=max_length
)

In [39]:
tokenized_inputs["input_ids"]

array([[ 4118, 19782,    27,   187,  2347,   476,   309,  7472,   253,
         3045,   285,  3290,   273,   253,  4561,  2505,   432,   418,
         4988,    74,  3210,    32,   187,   187,  4118, 37741,    27,
         2512,   403,  2067, 17082,   326,   476,   320,   908,   281,
         7472,   253,  3045,   285,  3290,   273,  4561,  2505,   432,
          418,  4988,    74,  3210,    13,  1690, 44229,   414,    13,
          378,  1843,    54,  4868,    13,   285,  1966,  7103,    15,
         3545, 12813,   414,  5593,   849,   973,   253,  1566, 26295,
          253,  1735,  3159,   275,   247,  3425,    13,  1223,   378,
         1843,    54,  4868,  5593,   253, 14259,   875,   253,  4561,
         2505,   285,   247,  3806,  2505,    15,  8801,  7103,  8687,
         1907,  1966, 16006,  2281,   253,  3290,   273,   253,  4561,
         2505,  1754,   327,  2616,   824,   347, 25253,    13,  2938,
         1371,    13,   285, 17200,    15,   733,   310,  8521,   281,
      

In [40]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [42]:
#finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")
df
tokenized_dataset = df.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

TypeError: tokenize_function() got an unexpected keyword argument 'batched'